# Problema:
## Medição de pressão numa coluna de um líquido com desidade variável.
* Nessa coluna está instalado barômetros medindo a pressão manométrica em 4 pontos diferentes.
<img src="t.png">
* Mas os engenheiros desejam desejam construir uma tubulação para escoar o líquido a uma profundidade de 2.7 metros abaixo do referencial de altura adotado. Para isso foi feita duas interpolações, com dois métodos diferentes, com os valores já conhecidos para estimar o valor da pressão naquele ponto.

## Método de Vandermonde

In [12]:
import numpy as np
from numpy.linalg import inv

#####  É importado a biblioteca numpy e um método para calcular a matriz inversa

In [11]:
def matriz_vm(tamanho,valores_x):
    '''
    Calcula a Matriz de Vandermonde
    '''
    matriz = []
    for i in range(len(valores_x)):
        linha = []
        for j in range(tamanho):
            valor = (valores_x[i])**j #Os valores são elevados ao número da coluna
            linha.append(valor)
        matriz.append(linha)
    return matriz

qntd_pontos = int(input("Digite a quantida de pontos: "))
valores_x = list(map(float,input("Digite os valores de x: ").split()))
A = matriz_vm(qntd_pontos,valores_x)
A_inv = inv(np.array(A))

for i in A:
    print(i)

Digite a quantida de pontos: 5
Digite os valores de x: -3 -1 0 2 3
[1.0, -3.0, 9.0, -27.0, 81.0]
[1.0, -1.0, 1.0, -1.0, 1.0]
[1.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 2.0, 4.0, 8.0, 16.0]
[1.0, 3.0, 9.0, 27.0, 81.0]


#####  A partir dos valores da quantidade de pontos e dos valores de x, foi possível calcular a matriz de Vandermonde

In [13]:
valores_y = list(map(float,input("Digite os valores de y: ").split()))
b = np.array(valores_y) #Transforma os valores de y em um array

for i in b:
    print(i)

Digite os valores de y: 35 14 7 0 -0.9
35.0
14.0
7.0
0.0
-0.9


#####  Com os valores da função foi possível construir a matriz 'b'

In [14]:
def gera_polinomio(coeficientes):
    '''
    Gera a string do polinomio
    '''
    polinomio = "%f + "%coeficientes[0]
    for i in range(1,len(coeficientes)):
        if i == len(coeficientes)-1:
            polinomio += "%fx^%i"%(coeficientes[i],i)
        else:
            polinomio += "%fx^%i + "%(coeficientes[i],i)
    return polinomio

coeficientes = list(np.dot(A_inv,b)) #Multiplica a inversa pelo vetor dos valores de y
polinomio = gera_polinomio(coeficientes)
print("p(x) = %s"%polinomio)

p(x) = 7.000000 + -5.758333x^1 + 1.229167x^2 + -0.025000x^3 + -0.012500x^4


#####  Multiplicando a inversa de 'A' por 'b', obtemos uma lista de coeficientes que podemos tranformar no polinomio

In [15]:
def interpolacao(coeficientes, x):
    '''
    Calcula o valor a ser interpolado no polinomio
    '''
    p_x = 0
    for i in range(len(coeficientes)):
        p_x += coeficientes[i] * (x**i) #Coeficiente multiplicado pelo valor elevado a potência correta
    return p_x

x = float(input("Digite o valor a ser interpolado: "))
p = interpolacao(coeficientes,x)
print("p(%f) = %f"%(x,p))

Digite o valor a ser interpolado: -2.7
p(-2.700000) = 31.335899


##### Aplicando o valor da interpolação no polinomio, calculamos o valor esperado

## Metodo de Newton

São importadas as bibliotecas que iremos utilizar

In [16]:
import numpy as np
from numpy.linalg import inv

##### É importado a biblioteca numpy

In [21]:
def gera_matriz(qntd_pontos,valores_y):
    matriz = []
    for i in range(qntd_pontos):
        linha = []
        for j in range(qntd_pontos):
            if j == 0:
                linha.append(valores_y[i])
            else:
                linha.append(0.0)
        matriz.append(linha)
    return matriz

qntd_pontos = int(input("Digite a quantida de pontos: "))
valores_y = list(map(float,input("Digite os valores de y: ").split()))
matriz = gera_matriz(qntd_pontos,valores_y)
for i in matriz:
    print(i)

Digite a quantida de pontos: 5
Digite os valores de y: 35 14 7 0 -0.9
[35.0, 0.0, 0.0, 0.0, 0.0]
[14.0, 0.0, 0.0, 0.0, 0.0]
[7.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0]
[-0.9, 0.0, 0.0, 0.0, 0.0]


A função acima gera a matriz com os valores de f(x) e é completa com os valores de 0
No primeiro laço de repetição i varia de 1 até a quantidade de pontos
no segundo laço j varia da mesma maneira de j

Perceba que a primeira coluna é composta com os valores de f(x)
A função retorna então a matriz formada com as operações anteriores.

In [22]:
vetor_x = list(map(float,input("Digite os valores de x: ").split()))
for i in vetor_x:
    print(i)

Digite os valores de x: -3 -1 0 2 3
-3.0
-1.0
0.0
2.0
3.0


Vetor com os valores de x

In [24]:
def calcula_f(qntd_pontos,vetor_x,matriz):
    for i in range(1,qntd_pontos): 
        for j in range(i,qntd_pontos):
            matriz[j][i] = ( (matriz[j][i-1]-matriz[j-1][i-1]) / (vetor_x[j]-vetor_x[j-i]))
    diagonal = list(np.diagonal(matriz))
    return diagonal

fs = calcula_f(qntd_pontos,vetor_x,matriz)
print(fs)

[35.0, -10.5, 1.1666666666666667, 0.0, -0.012500000000000002]


A função calcula_f que recebe a quantidade de pontos, o vetor com os pontos x e a matriz calculada na função anterior retorna 

In [25]:
def calcula_polinomio(qntd_pontos,fs,vetor_x,x):
    grau = qntd_pontos - 1
    p = fs[grau]
    for k in range(1,qntd_pontos):
        p = fs[grau-k] + (x -vetor_x[grau-k])*p
    return p

x = float(input("Informe o valor a ser interpolado: "))
p = calcula_polinomio(qntd_pontos,fs,vetor_x,x)
print("p(%f) = %f"%(x,p))

Informe o valor a ser interpolado: -2.7
p(-2.700000) = 31.335899


##### A função acima calcula o polinômio a partir dos pontos x e o ponto desejado da ser interpolado

Ela retorna p, que é o valor calculado a partir do polinômio gerado pelo metodo de newton

# Conclusão

* É possível concluir que os dois programas realizaram a interpolação corretamente e como segue o método e chegaram em resultados extremamente próximos.